In [23]:
#!/usr/bin/env python3

import unittest

from archemist.stations.syringe_pump_station.state import (
    SyringePumpStation,
    SyringePumpDispenseRateOp,
    SyringePumpDispenseVolumeOp,
    SyringePumpFinishDispensingOp,
)

from archemist.core.state.material import Liquid

from archemist.stations.syringe_pump_station.handler import APCSyringePumpStationRosHandler
from archemist.core.state.batch import Batch
from archemist.core.state.lot import Lot
from archemist.core.util.enums import StationState, OpOutcome
from archemist.core.state.station_op_result import ProcessOpResult, MaterialOpResult
from archemist.core.persistence.models_proxy import ModelProxy, ListProxy, DictProxy



from datetime import datetime

import time
from mongoengine import connect

print("importing done")


class syringPumpHandlerTest(unittest.TestCase):
    def setUp(self):
        self._db_name = 'archemist_test'
        self._client = connect(db=self._db_name, host='mongodb://localhost:27017', alias='archemist_state')

        self.station_doc = {
        'type': 'SyringePumpStation',
        'id': 20,
        'location': {'coordinates': [1,7], 'descriptor': "APCSyringePumpStation"},
        'total_lot_capacity': 1,
        'handler': 'APCSyringePumpStationHandler',
        'materials':
        {
            'liquids':
            [{
                'name': 'water',
                'amount': 400,
                'unit': 'mL',
                'density': 997,
                'density_unit': "kg/m3",
                "details": {"inlet_port": 4, "outlet_port": 7},
                'expiry_date': datetime.fromisoformat('2025-02-11')
            },
            {
                'name': 'acid',
                'amount': 400,
                'unit': 'mL',
                'density': 997,
                'density_unit': "kg/m3",
                "details": {"inlet_port": 1, "outlet_port": 2},
                'expiry_date': datetime.fromisoformat('2025-02-11')
            }]
        },
        'properties': None
    }


    def test_handler(self):
        station = SyringePumpStation.from_dict(self.station_doc)
        self.assertIsNotNone(station)
        self.assertEqual(station.state, StationState.INACTIVE)
            # construct lot
        batch = Batch.from_args(2)
        lot = Lot.from_args([batch])

        # add lot to station
        station.add_lot(lot)

        # construct handler
        handler = APCSyringePumpStationRosHandler(station)
        # initialise handler
        self.assertTrue(handler.initialise())

        # test IKAHeatStirBatchOp
        # construct op
        t_op = SyringePumpDispenseVolumeOp.from_args(
            target_sample= batch.samples[0],
            liquid_name= 'water', 
            dispense_volume= 25,
            dispense_unit= "mL",
            dispense_rate= 150,
            rate_unit=  "mL/minute"
        )



        self.assertIsNotNone(t_op.object_id)
        self.assertEqual(t_op.liquid_name, "water")
        self.assertEqual(t_op.dispense_volume, 25)
        self.assertEqual(t_op.dispense_unit, "mL")
        self.assertEqual(t_op.dispense_rate, 150)
        self.assertEqual(t_op.rate_unit, "mL/minute")

        station.add_station_op(t_op)
        station.update_assigned_op()
        self.assertEqual(station.assigned_op, t_op)
        self.assertEqual(station.assigned_op.liquid_name, "water")
        self.assertIsInstance(int(station.assigned_op.dispense_volume), int)
        self.assertIsInstance(station.liquids_dict, DictProxy)
        self.assertIsInstance(dict(station.liquids_dict), dict)
        # x = dict(station.liquids_dict)
        # print(x)
        print(station.liquids_dict)
        # print(station.liquids_dict.get(station.liquids_dict.))
        # # print(station.liquids_dict[station.assigned_op.liquid_name]['details'])
        liq = Liquid.from_object_id(station.liquids_dict[station.assigned_op.liquid_name].object_id)
        # self.assertIsInstance(liq, dict)
        details = liq.details
        print("inlet_port", details.get("inlet_port"))
        print("outlet_port", details.get("outlet_port"))


        # execute op
        handler._seq_id = 11

        handler.execute_op()

        # wait for results
        handler.is_op_execution_complete()

        self.assertEqual(handler.is_op_execution_complete(), False)

        while True:
            if handler.is_op_execution_complete():
                print("got the completion msg, moving on")
                break
            else:
                print("didn't get the completion msg, hence trying")
            time.sleep(2)

        self.assertEqual(handler.is_op_execution_complete(), True)



        outcome, op_results = handler.get_op_result()


        self.assertEqual(outcome, OpOutcome.SUCCEEDED)
        self.assertIsInstance(op_results[0], MaterialOpResult)


# if __name__ == '__main__':
#     unittest.main()


importing done


In [24]:
# Create a TestSuite
# Balance_suite = unittest.TestLoader().loadTestsFromTestCase(APCWeighingStationHandlerTest)

pump_suite = unittest.TestLoader().loadTestsFromTestCase(syringPumpHandlerTest)

# Run the tests
unittest.TextTestRunner().run(pump_suite)


[Batch-65b9172a286d8ed0f14db7f1]: (SyringePumpStation_20) stamp is added.
[SyringePumpStation_20]: Lot-65b9172a286d8ed0f14db7f4 is added for processing at slot 0
[SyringePumpStation_20]: <archemist.stations.syringe_pump_station.state.SyringePumpDispenseVolumeOp object at 0x7f7a5004a7f0> is added to queued_op list
inlet_port 4
outlet_port 7
[INFO] [1706628908.303226]: Sending Dispense Volume Command (volume : 25.0 mL of water)
didn't get the completion msg, hence trying
didn't get the completion msg, hence trying
didn't get the completion msg, hence trying
didn't get the completion msg, hence trying
didn't get the completion msg, hence trying
didn't get the completion msg, hence trying
didn't get the completion msg, hence trying
didn't get the completion msg, hence trying
didn't get the completion msg, hence trying
didn't get the completion msg, hence trying
didn't get the completion msg, hence trying
didn't get the completion msg, hence trying
didn't get the completion msg, hence tryin

.

got the completion msg, moving on



----------------------------------------------------------------------
Ran 1 test in 30.341s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>